# Marc Siquier Peñafort ASPMA LAB SESSIONS 2-3
[Task_definition](http://www.dtic.upf.edu/~aporter/amplab/)
## Task 2: Agreement between AcousticBrainz vs Million Song Database

Import all necessary functions

In [1]:
from utils import *
from evaluate import *
import numpy as np
import pandas as pd
from mir_eval.key import weighted_score as key_eval
from mir_eval.beat import f_measure as beat_eval

Define the location for Database files

In [2]:
AB_LOWLEVEL = 'Files/ab-2017-01-23-lowlevel-features.csv'
MSD_SUMMARY = 'Files/msd-2017-02-10-summary.json'
MBID_MSD = 'Files/msd-mbid-2016-01-results-ab.csv'

Import and process necessary data for BPM, DAN and KEY from each Dataset file.

__NOTE:__ _(Execution time: 1 minute)_

In [3]:
%%time
ab = import_ab(AB_LOWLEVEL)
msd = import_msd(MSD_SUMMARY, key_conf=0.6)
ids = import_ids(MBID_MSD)

Loading AB...
Processed 2352902 AB entries!!
Loading MSD...
MSD loaded!!
Processing...
Processed 237962 MSD entries!!
Loading IDS...
IDS loaded!!
Processing...
Processed 282046 IDS entries!!
CPU times: user 1min 17s, sys: 4.12 s, total: 1min 21s
Wall time: 1min 22s


We check if AB entries exist both in IDS file and in MSD Dataset.
If they exist both in AB and MSD we check that the duration is similar.

In [4]:
dur_th = 0.05 #precision window for duration
mix = check_duration(ab, msd, ids, dur_th)
print 'Common elements: %i' % len(mix)

Common elements: 212147


### AB bpm vs MSD tempo

####  Compute Accuracy 1
* Accuracy 1: The percentage of AB bpm estimates within 4% (the precision window) of the MSD tempo.

In [5]:
bpm_th = 0.04 #precision window for bpm

correct1 = 0
correct2 = 0
for d1, d2 in mix:
    correct1 += compare(d1['bpm'], d2['bpm'], bpm_th)
    correct2 += compare(d2['bpm'], d1['bpm'], bpm_th)

print "IF WE TAKE MSD AS GROUNDTRUTH:"
print "Correct instances:\t%i" % (correct1)
print "Total instances:\t%i" % (len(mix))
print "Accuracy:\t\t%.4f\n" % (correct1/float(len(mix)))
print "IF WE TAKE AB AS GROUNDTRUTH:"
print "Correct instances:\t%i" % (correct2)
print "Total instances:\t%i" % (len(mix))
print "Accuracy:\t\t%.4f\n" % (correct2/float(len(mix)))


IF WE TAKE MSD AS GROUNDTRUTH:
Correct instances:	136550
Total instances:	212147
Accuracy:		0.6437

IF WE TAKE AB AS GROUNDTRUTH:
Correct instances:	136571
Total instances:	212147
Accuracy:		0.6438



#### Compute Accuracy 2

* Accuracy 2: The percentage of AB bpm estimates within 4% of either the MSD tempo, or half, double, three times, or one third of the MSD tempo.

In [6]:
bpm_th = 0.04 #precision window for bpm

correct1 = 0
correct2 = 0
for d1, d2 in mix:
    correct1 += adv_compare(d1['bpm'], d2['bpm'], bpm_th)
    correct2 += adv_compare(d2['bpm'], d1['bpm'], bpm_th)

print "IF WE TAKE MSB AS GROUNDTRUTH:"
print "Correct instances:\t%i" % (correct1)
print "Total instances:\t%i" % (len(mix))
print "Accuracy:\t\t%.4f\n" % (correct1/float(len(mix)))
print "IF WE TAKE AB AS GROUNDTRUTH:"
print "Correct instances:\t%i" % (correct2)
print "Total instances:\t%i" % (len(mix))
print "Accuracy:\t\t%.4f\n" % (correct2/float(len(mix)))

IF WE TAKE MSB AS GROUNDTRUTH:
Correct instances:	162810
Total instances:	212147
Accuracy:		0.7674

IF WE TAKE AB AS GROUNDTRUTH:
Correct instances:	162831
Total instances:	212147
Accuracy:		0.7675



### AB key_key/key_scale vs key/mode
* consider different confidence levels provided by MSD (key_confidence, mode_confidence) in evaluation
* use [mir_eval library](https://craffel.github.io/mir_eval/#module-mir_eval.key)


In [7]:
score1 = []
score2 = []
for d1, d2 in mix:
    if d2['key']:
        score1.append(key_eval(d1['key'], d2['key']))
        score2.append(key_eval(d2['key'], d1['key']))
print "IF WE TAKE MSD AS GROUNDTRUTH:"
print "Total instances:\t%i" % (len(score1))
print "Score:\t\t\t%.4f\n" % (sum(score1)/float(len(score1)))
print "IF WE TAKE AB AS GROUNDTRUTH:"
print "Total instances:\t%i" % (len(score2))
print "Score:\t\t\t%.4f\n" % (sum(score2)/float(len(score2)))

IF WE TAKE MSD AS GROUNDTRUTH:
Total instances:	45100
Score:			0.6248

IF WE TAKE AB AS GROUNDTRUTH:
Total instances:	45100
Score:			0.5950



## <span style="color:blue">IN ORDER TO CONTINUE WITH BEATS EVALUATION:

## <span style="color:BLUE">RESTART THE KERNEL AND RE-RUN CELL 1 (IMPORTS)
### AB beats_position vs MSD beats_start 
* consider different confidence levels provided by MSD (beats_confidence) in evaluation
* use [mir_eval library](https://craffel.github.io/mir_eval/#module-mir_eval.beat)

Define the location for all needed dataset files

In [4]:
MBID_MSD = 'Files/msd-mbid-2016-01-results-ab.csv'
AB_BEATS = 'Files/ab-2017-01-23-beats.csv'
MSD_BEATS = 'Files/msd-2017-02-10-beats-rounded.csv'

Import and process necessary data for BEATS evaluation from each Dataset file.

__NOTE:__ _(Execution time: 6 minutes)_

In [ ]:
%%time
ids = import_ids(MBID_MSD)
ab_beats = import_ab_beats(AB_BEATS)
msd_beats = import_msd_beats(MSD_BEATS, beats_conf=0.7)

idskeys = set(ids.keys())
msdkeys = set(msd_beats.keys())
score1 = []
score2 = []
for mbid in ab_beats:
    if mbid in idskeys:
        msdk = ids[mbid]
        if msdk in msdkeys:
            b1 = ab_beats[mbid]
            b2 = np.array(msd_beats[msdk])
            score1.append(beat_eval(b1, b2))
            score2.append(beat_eval(b2, b1))

print 'IF WE TAKE MSD AS GROUND TRUTH'
print "Total instances:\t%i" % (len(score1))
print 'F_score:\t\t%.4f\n' % (sum(score1) / float(len(score1)))

print 'IF WE TAKE AB AS GROUND TRUTH'  
print "Total instances:\t%i" % (len(score2))
print 'F_score:\t\t%.4f\n' % (sum(score2) / float(len(score2)))

Loading IDS...
IDS loaded!!
Processing...
Processed 282046 IDS entries!!
Loading AB beats...
Processed 282046 AB beats instances...
Loading MSD beats...
Processed 237962 MSD beats instances...


/usr/local/lib/python2.7/dist-packages/mir_eval/beat.py:93: UserWarning: Estimated beats are empty.
  warnings.warn("Estimated beats are empty.")
/usr/local/lib/python2.7/dist-packages/mir_eval/beat.py:91: UserWarning: Reference beats are empty.
  warnings.warn("Reference beats are empty.")
